In [3]:
import pickle
import numpy as np

from mofnet import DataLoader, MOFNet

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.problems import get_problem
from pymoo.visualization.scatter import Scatter
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem import Problem
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.optimize import minimize
import pymoo.gradient.toolbox as anp

2023-07-23 14:36:22.363841: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
data_loader = DataLoader.from_state("data_loader_state-20200116.npz")

# This hash dicts map budilding block names to unique index.
topo_hash = data_loader.topo_hash
node_hash = data_loader.node_hash
edge_hash = data_loader.edge_hash

mofnet = MOFNet()
mofnet.initialize_weights()
training_name = "cycle_tot"
mofnet.load_weights("cycle_tot/mofnet-{n:}-min.h5".format(n=training_name))

2023-07-23 14:36:33.736080: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


In [5]:
mofnet_n2 = MOFNet()
mofnet_n2.initialize_weights()
training_name = "cycle_tot"
mofnet_n2.load_weights("cycle_tot_n2/mofnet-{n:}-min.h5".format(n=training_name))

In [6]:
def mofnet_prediction(x_test_name):
    test_bulk=data_loader.make_dataset(
        np.array(x_test_name, dtype=str),
        np.array([0.0]*len(x_test_name)),
        batch_size=10000,
        repeat=False,
        shuffle=False,
    )
    prediction=[]
    for x, y in test_bulk:
        prediction.append(mofnet(x)*50)
    return prediction[0]

In [7]:
def mofnet_n2_prediction(x_test_name):
    test_bulk=data_loader.make_dataset(
        np.array(x_test_name, dtype=str),
        np.array([0.0]*len(x_test_name)),
        batch_size=10000,
        repeat=False,
        shuffle=False,
    )
    prediction=[]
    for x, y in test_bulk:
        prediction.append(mofnet_n2(x))
    return prediction[0]

In [8]:
def get_mof_name(topo,avail_nbbs,n_ebbs,variables_list):
    name_list=[]
    for variables in variables_list:
        name=topo
        for i in range(len(avail_nbbs)):
            name=name+"+"+avail_nbbs[i][2][variables[i]]
        for j in range(len(avail_nbbs),len(avail_nbbs)+n_ebbs):
            name=name+"+"+avail_ebbs[2][variables[j]]
        name_list.append(name)
    return name_list

In [9]:
def get_mof_name_independent(topo,x):
        topo_cn=topo_cn_arr[topo_cn_arr[:,0]==topo][0] #connection profile of each bbs in the topo
        n_bbs=topo_cn[2][0]+topo_cn[2][1] #unique nbb amounts in the topo
        n_ebbs=topo_cn[2][1] #unique ebb amounts in the topo
        avail_nbbs=[]
        xu_list=[]
        for nbb_cn in topo_cn[1]:
            nbb=bb_cn_arr[bb_cn_arr[:,0]==nbb_cn][0]
            avail_nbbs.append(nbb)
        for avail_nbb in avail_nbbs:
            xu_list.append(avail_nbb[1]-1)
        for avail_ebb in range(n_ebbs):
            xu_list.append(avail_ebbs[1]-1)
        return get_mof_name(topo,avail_nbbs,n_ebbs,x)

In [10]:
#global variables
with open('topo_cn_arr.pickle', 'rb') as file:
    topo_cn_arr = pickle.load(file)
with open('bb_cn_arr.pickle', 'rb') as file:
    bb_cn_arr = pickle.load(file)
avail_ebbs=bb_cn_arr[bb_cn_arr[:,0]==2][0]
avail_topos=topo_cn_arr[:,0]
#np.savetxt("avail_topos.txt",avail_topos,fmt="%s")
topo_cn_arr,bb_cn_arr,avail_ebbs,avail_topos #[name,nbb_types,n_nbbs&ebbs],[cn,n_bbs,bbs_list]

(array([['abf', list([4, 3, 4]), list([3, 3])],
        ['abr', list([4, 4, 4]), list([3, 3])],
        ['acs', list([6]), list([1, 1])],
        ...,
        ['xbh', list([6, 4]), list([2, 2])],
        ['urp', list([8, 12, 3]), list([3, 2])],
        ['twf-e', list([6]), list([1, 1])]], dtype=object),
 array([[2, 220,
         list(['E0', 'E226', 'E50', 'E162', 'E139', 'E212', 'E51', 'E19', 'E27', 'E44', 'E182', 'E127', 'E102', 'E173', 'E63', 'E70', 'E158', 'E132', 'E108', 'E146', 'E69', 'E184', 'E138', 'E168', 'E115', 'E88', 'E79', 'E217', 'E128', 'E52', 'E58', 'E92', 'E48', 'E216', 'E67', 'E165', 'E204', 'E175', 'E202', 'E66', 'E131', 'E95', 'E71', 'E15', 'E37', 'E13', 'E166', 'E72', 'E18', 'E73', 'E167', 'E205', 'E228', 'E206', 'E186', 'E185', 'E220', 'E89', 'E149', 'E180', 'E76', 'E171', 'E151', 'E43', 'E84', 'E119', 'E25', 'E29', 'E14', 'E45', 'E16', 'E57', 'E199', 'E159', 'E85', 'E192', 'E120', 'E130', 'E195', 'E26', 'E201', 'E156', 'E143', 'E211', 'E197', 'E142', 'E160', 'E153

In [11]:
class carbon_problem(Problem):

    def __init__(self,topo_name):  
        self.topo=topo_name 
        self.topo_cn=topo_cn_arr[topo_cn_arr[:,0]==self.topo][0] #connection profile of each bbs in the topo
        self.n_bbs=self.topo_cn[2][0]+self.topo_cn[2][1] #unique nbb amounts in the topo
        self.n_ebbs=self.topo_cn[2][1] #unique ebb amounts in the topo
        self.avail_nbbs=[]
        self.xu_list=[]
        for nbb_cn in self.topo_cn[1]:
            nbb=bb_cn_arr[bb_cn_arr[:,0]==nbb_cn][0]
            self.avail_nbbs.append(nbb)
        for avail_nbb in self.avail_nbbs:
            self.xu_list.append(avail_nbb[1]-1)
        for avail_ebb in range(self.n_ebbs):
            self.xu_list.append(avail_ebbs[1]-1)
        super().__init__(n_var=self.n_bbs, n_obj=2, n_ieq_constr=0, vtype=int)
        self.xl = np.zeros(self.n_var).astype(int)
        self.xu = np.array(self.xu_list)
        print(self.topo_cn,self.xu)

    def _evaluate(self, x, out, *args, **kwargs):
        self.mof_name_list=get_mof_name(self.topo,self.avail_nbbs,self.n_ebbs,x)
        f1 = -mofnet_prediction(self.mof_name_list)[:,0]
        f2 = mofnet_n2_prediction(self.mof_name_list)[:,0]


        out["F"] = anp.column_stack([f1, f2])

    def _calc_pareto_front(self, n_points=100):
        initial_pop=[]
        for i in range(self.n_var):
            x0=np.linspace(0, self.xu[i], n_points, dtype=int)
            initial_pop.append(x0)
            
        X = np.column_stack(initial_pop)
        return self.evaluate(X, return_values_of=["F"])

In [12]:
algorithm = NSGA2(pop_size=100,
                      sampling=IntegerRandomSampling(),
                        crossover=SBX(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
                        mutation=PM(prob=0.5, eta=3.0, vtype=float, repair=RoundingRepair()))

In [ ]:
for target_topo in avail_topos:
    problem = carbon_problem(target_topo)
    res = minimize(problem,
                       algorithm,
                       termination=('n_gen', 500),
                       seed=1,
                       save_history=True,verbose=True)
    #plot = Scatter()
    #plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
    #plot.add(res.F, color="red")
    #plot.show()
    final_mofs=get_mof_name_independent(target_topo,res.X)
    final_results=np.vstack((get_mof_name_independent(target_topo,res.X),res.F.T)).T #[name,q_co2,lg(q_n2)]
    np.savetxt("nsga_result/"+target_topo+".txt",final_results,fmt="%s")
    del problem

['xff' list([7]) list([1, 1])] [  4 219]


2023-07-23 14:37:17.990640: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]
2023-07-23 14:37:18.629105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]
2023-07-23 14:37:19.157996: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]


2023-07-23 14:37:19.704520: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]


n_gen  |  n_eval  | n_nds  |      igd      |       gd      |       hv     
     1 |       98 |      6 |  0.3009763530 |  0.0838095429 |  0.6936350946


2023-07-23 14:37:20.265196: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]
2023-07-23 14:37:20.640501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]


...

2023-07-23 14:44:16.737112: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]
2023-07-23 14:44:17.101494: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]


    56 |     5598 |     20 |  0.3107952494 |  0.0739777624 |  0.6968195050


2023-07-23 14:44:17.481368: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]
2023-07-23 14:44:17.796116: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]


    57 |     5698 |     20 |  0.3107952494 |  0.0739777624 |  0.6968195050


2023-07-23 14:44:18.211198: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]
2023-07-23 14:44:18.563465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int32
	 [[{{node Placeholder/_7}}]]


array(['xfd', 'xff', 'xfg', 'xfh', 'xfi', 'xfj', 'xfk', 'xfo', 'xfp',
       'xfq', 'xft', 'xfu', 'xfw', 'xfx', 'xfy', 'xfz', 'xij', 'xly',
       'xlz', 'xmz', 'xux', 'xww', 'xxv', 'yan', 'yav', 'yav-d', 'ybh',
       'yfa', 'yfb', 'yfc', 'yfd', 'yfe', 'yff', 'yfg', 'yfh', 'yfi',
       'yfj', 'yfk', 'yfl', 'yfm', 'yfn', 'yfo', 'yfq', 'yfr', 'yfs',
       'yft', 'yfu', 'yfw', 'yfx', 'yfy', 'yfz', 'ylf', 'yug', 'zba',
       'zbd', 'zbf', 'zbh', 'zbl', 'zbs', 'zbt', 'zbw', 'zec', 'zfa',
       'zfb', 'zfc', 'zfd', 'zfe', 'zff', 'zfg', 'zfh', 'zfi', 'zfj',
       'zfk', 'zfl', 'zfm', 'zfn', 'zfo', 'zfp', 'zfq', 'zfr', 'zfs',
       'zft', 'zfu', 'zfv', 'zfw'], dtype=object)